# Import necessary modules

In [30]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
dictionary_words = set(nltk.corpus.words.words())

from pprint import pprint

from textblob import TextBlob, Word

import spacy
import spacy_legacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models

import itertools 
#custom list of stop words
stop_words_english = []
with open('stop_words_english.txt', encoding="utf8") as my_file:
    for line in my_file:
        stop_words_english.append(line.replace("\n", ""))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Ber

#  Read the dataset

In [31]:
df = pd.read_csv ('AllReviews0120-1122.csv')

In [32]:
df.head()

,ReviewID,Page Type,Full Text,Rating
0,1,review,"When they work they are great, however, in my ...",1
1,2,review,I have a garmin Vivosmart 4 and this fitbit ch...,1
2,3,review,Absolute Trash! Seriously felt like I bought t...,1
3,4,review,I got this to pair with my apple watch and cha...,1
4,5,review,I am very disappointed with the FitBit Charge ...,1


In [33]:
df.shape

(35895, 4)

In [34]:
df= df.apply(lambda x: x.astype(str).str.lower())

In [35]:
reviews = np.array(df['Full Text'])
reviews = ' '.join(reviews)
#reviews

# Preprocess reviews

In [36]:
def clean_text(docs):
    for i in range(len(docs)):
        #remove hashtags and mentions
        docs[i] = " ".join(filter(lambda x:x[0]!='#', docs[i].split())) #remove hashtags
        docs[i] = " ".join(filter(lambda x:x[0]!='@', docs[i].split())) #remove mentiones
        
        #remove special chars, anything not a number or letter, anything excpet alphanumeric words
        #docs[i] = re.sub(r'[^\w\s]','',docs[i]) #remove punctuation marks and special chars
        #docs[i] = re.sub('[^A-Za-z0-9]+', ' ', docs[i]) #remove anything that is not a number or or letter
        #docs[i] = re.sub('[\W_]', ' ', docs[i]) #Remove any thing excpet charcater and alphanumeric words
        #docs[i] = re.sub(r'\w*\d\w*', '', docs[i]) #Remove any thing with numbers
        #docs[i] = ' '.join(word for word in docs[i].split() if not word.isdigit())
        
        #compare words to englisg dictionary and remove nonsense words
        #words = [word for word in docs[i].split() if word.lower() in dictionary_words or not word.isalpha()]
        #docs[i] = " ".join(words)
        
        #remove words that consists of less than three letter
        #words = [word for word in docs[i].split() if len(word)>2]
        #docs[i] = " ".join(words)
                             
        #Remove stop words using custom list of stop words
        #words = [word for word in reviews[i].split() if word.lower() not in stop_words_english]
        #reviews[i] = " ".join(words)

        #Remove brands related keywords
        
        
        brands2kw = ["fitbit ace", "fitbit charge", "fitbit verca", "fitbit ionic",  "fitbit versa lite",  
                  "fitbit inspire",  "fitbit versa",  "garmin forerunner",  "garmin fēnix",  "garmin vívoki",  
                  "garmin vívofit",  "garmin vívoactive",  "garmin vívomove",  "garmin vívosmart",  
                  "garmin vívosport",  "garmin venu",  "garmin instinct",  "garmin luxe",  "garmin darth vader",  
                  "garmin captain marvel",  "garmin approach",  "garmin lily",  "garmin swim",  "xiaomi mi band",  
                  "xiaomi mi samrt band",  "moov now",  "moov hr",  "apple watch",  "fossil gen",  
                  "fossil sport smartwatch",  "fossil hybrid smartwatch hr",  "misfit vap, ",  "misfit ray",  
                  "misfit path",  "misfit command",  "withings steel",  "withings move",  "withings pulds",  
                  "withings move ecg",  "ihealth watch",  "samsung gear",  "samsung galaxy watch active",  
                  "samsung galaxy fit",  "polar a360",  "polar a370",  "polar m430",  "polar m200",  "polar h10",  
                  "polar vantage",  "polar ignite",  "polar grit x",  "polar titan",  "polar oh1",  "polar h9",  
                  "striiv fusion",  "striiv apex hr",  "striiv dash hr",  "huawei talkband",  " huawei watch",  
                  "huawei band",  "mykronoz zewatch",  "mykronoz zefit",  "mykronoz zesport",  "mykronoz zetime",  
                  "mykronoz zefit",  "mykronoz zeround",  "mykronoz zeneo",  "mykronoz zetrack",  "coros apex",  
                  "wyze band",  "letsfit fitness racker",  "coros pace",  "withings scanwatch",  "amazon halo",  
                  "timex ironman",  "suunto peak",  "wahoo fitness"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands2kw]
        docs[i] = ' '.join(resultwords)
        
        brands1kw = ["fitbit", "garmin", "apple", "xiaomi", "moov", "fossil", "misfit", "withings", 
                     "ihealth", "samsung", "polar", "striiv", "huawei", "mykronoz", "coros", "wyze", 
                     "letsfit", "amazon"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands1kw]
        docs[i] = ' '.join(resultwords)
        

        #Remove feature-related words
        features = ["ability", "quality", "feature", "aspect", "abilities", "qualities", "features", "aspects"]
        resultwords  = [word for word in docs[i].split() if word.lower() not in features]
        docs[i] = ' '.join(resultwords)

    return docs

In [37]:
reviews = np.array(df['Full Text'])
processed_reviews = clean_text(reviews)

# Define a function to split each review using punctuations and conjunctions

In [38]:
def replace_punctuations(string):
    punctuations = ['.', ',', '&', ';', '!', ':']
    for punctuation in punctuations:
        string = string.replace(punctuation, '-')
    return string

In [39]:
for i in range(len(processed_reviews)):
    processed_reviews[i] = replace_punctuations(processed_reviews[i]) 

In [40]:
def replace_keywords(string, keywords):
    words = string.split()
    for i, word in enumerate(words):
        if word in keywords:
            words[i] = '-'
    return ' '.join(words)

In [41]:
keywords = ['for', 'and', 'but', 'or', 'so', 'after', 'as', 'because', 'before', 'even', 'if', 'now', 'once', 'since', 'than', 'that', 'thought', 'when', 'where', 'which', 'while', 'who', 'what']
for i in range(len(processed_reviews)):
    processed_reviews[i] = replace_keywords(processed_reviews[i], keywords) 

In [42]:
def partition_string(string):
    return string.split("-")

In [43]:
for i in range(len(processed_reviews)):
    processed_reviews[i] = partition_string(processed_reviews[i]) 

In [44]:
df1 = pd.DataFrame(processed_reviews)
df1.columns = ['Full Text']
df1.head()

,Full Text
0,"[, they work they are great, however, in my..."
1,"[i have a vivosmart 4 , this charge 4, howev..."
2,"[absolute trash, seriously felt like i bought..."
3,"[i got this to pair with my watch , chart my ..."
4,"[i am very disappointed with the charge 3, i ..."


In [45]:
df1 = df1.join(df['Rating'])

In [46]:
df1

,Full Text,Rating
0,"[, they work they are great, however, in my...",1
1,"[i have a vivosmart 4 , this charge 4, howev...",1
2,"[absolute trash, seriously felt like i bought...",1
3,"[i got this to pair with my watch , chart my ...",1
4,"[i am very disappointed with the charge 3, i ...",1
...,...,...
35890,"[love it, i have a very small wrist , the fa...",5
35891,"[i loved this , better , my watch , it only...",5
35892,[watch series 3 (gps) 38mm silver aluminum cas...,5
35893,"[i love my watch, helps me keep track of my f...",5


# Export partitioned reviews for validation

In [47]:
dfValidation = pd.read_csv ('AllReviews0120-1122.csv')
dfValidation["Partitioned Review"] = ""
dfValidation["Partitioned Review"] = df1["Full Text"]
dfValidation.head()
dfValidation.to_csv('PartitionedReviewForValidation.csv')

# Functions to check sentiment, quality terms, negators, and context terms 

In [48]:
# # # test string matching using regular expressions
# import re
# check = 0
# check = re.search(r"\b"+'dish'+r"\b","this is broken dishwasher")
# if check == None:
#     print('No')
# else:
#     print('Yes')



In [49]:
############## Check Irrelevant Review Partitions ####################
def checkirrelevant(string):
    filterwords = ["band", "bands", "case", "cases", "cover", "protective", "protector", "shield", 
                   "appltv", "gear vr", "tv", "shoe", "mountain", "mountains", "beach", "appltv", 
                   "accessory", "accessories", "charger"]
    checkirr = False
    for i in range(len(filterwords)):
        if ((re.search(r"\b"+filterwords[i]+r"\b",string) != None)): 
            checkirr = True
    return checkirr
 
#checkirrelevant('the band looks good')  # return true if there is irrelevant words, otherwise false

In [50]:
############## Check Sentiment ####################

def checkSent(string):
    return sentiment_pipeline(string)[0].get('label') #returns POSITIVE or NEGATIVE as a string

data = "I love you"
checkSent(data)

'POSITIVE'

In [51]:
############## Check Terms ####################

terms = ['easy to use', 'simple', 'straightforward', 'user-friendly', 'intuitive']

def checkterms(string, termsvalues):
    checkterm = False
    for i in range(len(termsvalues)): #now can do exact matching
        if ((re.search(r"\b"+termsvalues[i]+r"\b",string) != None)): 
            checkterm = True
    return checkterm
 
#checkterms('This is hello', terms)   # return true if a quality term was found, otherwise, false

In [52]:
############## Check Negators ####################
def checknegators(string):
    negatorvalues = ["ain't", "aren't", "cannot", "cant", "can't", "couldn't", "daren't", "didn't", 
                     "don't", "hasn't","haven't", "isn't", "mayn't","mightn't","mustn't","needn't",
                     "not","oughtn't","shan't","shouldn't","weren't", "won't", "wouldn't"]
    checknegator = False
    for i in range(len(negatorvalues)):
        if ((re.search(r"\b"+negatorvalues[i]+r"\b",string) != None)):
            checknegator = True
    return checknegator
 
#checknegators('This is not hello simple') # return true if a negator term was found, otherwise, false.

In [53]:
############## Check Context ####################

def checkcontext(string, contextvalues):
    checkcontext = False
    for i in range(len(contextvalues)):
        if ((re.search(r"\b"+contextvalues[i]+r"\b",string) != None)):
            checkcontext = True
    return checkcontext
 
#terms = ['health activities','target']   
#checkterms('I use wearables to achieve target goals', terms)
#return true iof a context term was found, otherwise, false.

# Define fucntions to encode reviews


In [54]:
############## Encode Reviews v1 without Context terms  'encodereviews' ####################

In [55]:
#myArray = ['easy to use watch', 'health ', 'attention']

#ease_of_use_terms = ['simple', 'easy to use',  'straightforward', 'user-friendly', 'intuitive']

# ['this replacement band for the charge 3 seemed great at the start', ' but the two loops broke within 6 months of use', ' in this case', " the cheap price wasn't worth it"]
def encodereviews(reviewarray, qualityterms):
    encode = 0
    for i in reviewarray:
        if(((checknegators(i) == False) or (checkSent(i) == 'POSITIVE')) and # no negatives, not negative sentiment
           (checkterms(i, qualityterms) == True) and                         # has a quality dimension term 
           (checkirrelevant(i) == False)):                                   # has no irrelevant terms
            encode = 1
        else:
            continue
    return encode   

#encodereviews(myArray, ease_of_use_terms)

In [56]:
############## Encode Reviews v2 including Context terms  'encodereviewswc' ####################


In [57]:
def encodereviewswc(reviewarray, qualityterms, contextterms):
    encode = 0
    for i in reviewarray:
        if(((checknegators(i) == False) or (checkSent(i) == 'POSITIVE')) and   # no negatives, not negative sentiment
           (checkterms(i, qualityterms) == True) and                           # has a quality dimension term
           (checkcontext(i, contextterms) == True) and                         # has a context term
           (checkirrelevant(i) == False)):                                     # has no irrelevant terms
            encode = 1
        else:
            continue
    return encode   

# Encode Reviews

In [58]:
df1["Hedonic Motivation"] = ""

terms = ['enjoyable', 'enjoyment', 'enjoy', 'pleasant', 'flow experience', 'pleasure', 'fun', 
         'interesting', 'satisfied', 'satisfaction', 'feel good', 'happiness', 'happy', 'happier', 
         'entertainment', 'entertain', 'exciting', 'playfulness', 'playful',  'feeling of accomplishment']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Hedonic Motivation'] = 1
    else:
        df1.at[i,'Hedonic Motivation'] = 0
        

df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation
0,"[, they work they are great, however, in my...",1,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0
2,"[absolute trash, seriously felt like i bought...",1,1
3,"[i got this to pair with my watch , chart my ...",1,0
4,"[i am very disappointed with the charge 3, i ...",1,0


In [59]:
df1["Connectivity Support"] = ""

terms = ['pair', 'sync', 'connect']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Connectivity Support'] = 1
    else:
        df1.at[i,'Connectivity Support'] = 0
    
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support
0,"[, they work they are great, however, in my...",1,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0
2,"[absolute trash, seriously felt like i bought...",1,1,0
3,"[i got this to pair with my watch , chart my ...",1,0,1
4,"[i am very disappointed with the charge 3, i ...",1,0,0


In [60]:
df1["Customizability"] = ""

terms = ['customize', 'custom', 'personalize', 'personalization']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Customizability'] = 1
    else:
        df1.at[i,'Customizability'] = 0
        
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability
0,"[, they work they are great, however, in my...",1,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0
2,"[absolute trash, seriously felt like i bought...",1,1,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0


In [61]:
df1["Perceived Ease of Use"] = ""

terms = ['easy', 'clear', 'understandable', 'easy to use', 'mental effort', 
         'trouble free', 'simple', 'controllable', 'easy to learn']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Perceived Ease of Use'] = 1
    else:
        df1.at[i,'Perceived Ease of Use'] = 0
        
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use
0,"[, they work they are great, however, in my...",1,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0


In [62]:
df1["Appeal"] = ""

terms = ['cool', 'appearance', 'aesthetic', 'stylish', 'fashion', 'accessory', 'pleasing aesthetics', 'color', 
         'design', 'texture', 'uniqueness', 'unique', 'size', 'look']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Appeal'] = 1
    else:
        df1.at[i,'Appeal'] = 0
        
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal
0,"[, they work they are great, however, in my...",1,0,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0,0
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0,0
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0,0


In [63]:
df1["Perceived Value"] = ""

terms = ['price', 'value', 'cost', 'monetary', 'money']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Perceived Value'] = 1
    else:
        df1.at[i,'Perceived Value'] = 0

df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value
0,"[, they work they are great, however, in my...",1,0,0,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0,0,0
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0,0,0
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0,0,0


In [64]:
# sleep tracking	648       # track of	488    # does everything	439   # rate monitor	382
# ability to	370    # working out	359     # blood pressure	326     # keep track	284
# fitness tracking	269    It tracks	155    everything I need	202    #keep track of	267
# my heart rate	405      # to keep track	135    # to track my	411     # use it for	181
# use it to	134  # was able to	317 # and heart rate	69 # everything I need it	79 #has all the features	56
# heart rate	57   #it to track my	64  #keep track of my	134  # steps	71   #helps me keep track of	25   
#I use it every day	25    #I use it to track	24   track my heart rate	24    track my steps and	33

df1["Perceived Usefulness"] = ""

terms = ['rejuvenated', 'simplify', 'control over', 'reduce', 'accomplish more', 'save time', 'work more quickly',
        'accomplish','accomplished','accomplishes','accomplishing','achieve','achieved','achieves','achieving',
        'advantageous ','beneficial','check','checked','checking','checks','complete','completes','completing',
        'completing','convenient','conveniently','develop','developed','developing','develops','effective',
        'effectively','effectiveness','enable','enables','enhance','enhanced','enhances','enhancing','handle',
        'handled','handles','help','helpful','helps','improve','improved','improves ','improving','increase',
        'increased','increases','increasing','maintain','maintained ','maintaining','manage','managed','manages',
        'managing','monitor','monitored','monitoring','monitors','obtain','obtained','obtaining','obtains',
        'perform','performed','performing','performs','provide','provided','provides','providing','reduce',
        'reduced','reduces','reducing','save','saved','saves','saving','useful', 'track', 'does']


context = ['act','activities','concerns','convenience','daily activities','daily health','daily healthcare',
           'daily life','efficiency','effort','exercise','fitness','goals','good features','health condition',
           'health information','health management','health monitoring','healthcare','healthcare activities',
           'healthy habits','healthy status','information','job','learning achievement','life',
           'life more convenient ','life more effective','managing my health','managing my health',
           'meeting my needs','meeting my needs','performance','physical activities','physical health',
           'productivity','quality','quality of my life','safety','service','stress','task','tasks','things',
           'things more quickly','time','work satisfaction','activity','acts','concern','exercises','goal',
           'good feature','health conditions','healthcare activity','healthy habit','jobs','manage health',
           'manage my heakth','managing health','meeting need','meeting needs','physical activity','quality of life',
           'services','task','tasks','manages my health','meet needs','manages health','meet need', 'steps','calories',
           'distance','floors','active minutes','sleep time','sleep quality', 'heart rate', 'everything']



for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviewswc(df1['Full Text'][i], terms, context) == 1):
        df1.at[i,'Perceived Usefulness'] = 1
    else:
        df1.at[i,'Perceived Usefulness'] = 0
        
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness
0,"[, they work they are great, however, in my...",1,0,0,0,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0,0,0,0
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0,0,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0,0,0,0
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0,0,0,0


In [65]:
df1["Device Quality"] = ""

terms = ['quality', 'system quality', 'service quality', 'battery life', 'charge', 'long time', 'battery last']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Device Quality'] = 1
    else:
        df1.at[i,'Device Quality'] = 0
        
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality
0,"[, they work they are great, however, in my...",1,0,0,0,0,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0,0,0,0,1
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0,0,0,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0,0,0,0,0
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0,0,0,0,1


In [66]:
df1["Credibility Support"] = ""

terms = ['accuracy', 'accurate']

for i in range(len(df1['Full Text'])):
    if(i%1000 == 0):
        print(i)
    if(encodereviews(df1['Full Text'][i], terms) == 1):
        df1.at[i,'Credibility Support'] = 1
    else:
        df1.at[i,'Credibility Support'] = 0
        
df1.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support
0,"[, they work they are great, however, in my...",1,0,0,0,0,0,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0,0,0,0,1,1
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0,0,0,0,0,0
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0,0,0,0,0,1
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0,0,0,0,1,0


In [67]:
df2 = df1
df2["Sum"] = ""

In [68]:
for i in range(len(df2['Full Text'])):
    df2['Sum'][i] = df2['Perceived Ease of Use'][i] + df2['Hedonic Motivation'][i] + \
                df2['Connectivity Support'][i] + df2['Customizability'][i] + df2['Appeal'][i] + \
                df2['Perceived Value'][i] + df2['Perceived Value'][i] + df2['Perceived Usefulness'][i] + \
                df2['Device Quality'][i] + df2['Credibility Support'][i]

In [69]:
df2.head()

,Full Text,Rating,Hedonic Motivation,Connectivity Support,Customizability,Perceived Ease of Use,Appeal,Perceived Value,Perceived Usefulness,Device Quality,Credibility Support,Sum
0,"[, they work they are great, however, in my...",1,0,0,0,0,0,0,0,0,0,0
1,"[i have a vivosmart 4 , this charge 4, howev...",1,0,0,0,0,0,0,0,1,1,2
2,"[absolute trash, seriously felt like i bought...",1,1,0,0,0,0,0,0,0,0,1
3,"[i got this to pair with my watch , chart my ...",1,0,1,0,0,0,0,0,0,1,2
4,"[i am very disappointed with the charge 3, i ...",1,0,0,0,0,0,0,0,1,0,1


In [70]:
df2.shape

(35895, 12)

In [71]:
df2 = df2[df2.Sum != 0]
df2.shape

(19360, 12)

In [72]:
df2.to_csv('ProcessedReviewsv5.csv')